# [Diffusion Convolutional Recurrent Neural Network: Data-Driven Traffic Forecasting](https://openreview.net/pdf?id=SJiHXGWAZ)

##  Tensorflow LSTM cell analysis only, but it is the fundamental part for implementing this literature

### 1 [How does lstm work in TensorFlow ?](https://stackoverflow.com/questions/54767816/how-exactly-does-lstmcell-from-tensorflow-operates)

In [46]:
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
num_units = 2
lstm = tf.nn.rnn_cell.LSTMCell(num_units = num_units)
batch=1
timesteps = 7
num_input = 4
X = tf.placeholder("float", [batch, timesteps, num_input])
x = tf.unstack(X, timesteps, 1) # the same as [tf.reshape(X,(-1,num_input))]
outputs, states = tf.contrib.rnn.static_rnn(lstm, x, dtype=tf.float32)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
x_val = np.reshape(range(28),[batch, timesteps, num_input])
res = sess.run(outputs, feed_dict = {X:x_val})
for e in res:
    print(e)
print("\nmy imp\n")
#my impl
def sigmoid(x):
    return 1/(1+np.exp(-x))

kernel,bias=sess.run([lstm._kernel,lstm._bias]) # LSTM 内部权重矩阵和偏置
f_b_=lstm._forget_bias
c,h=np.zeros([batch,num_input-1]),np.zeros([batch,num_input-1])
for step in range(timesteps):
    inpt=np.split(x_val,7,1)[step][0]
    lstm_mtrx=np.matmul(np.concatenate([inpt,h],1),kernel)+bias
    i,j,f,o=np.split(lstm_mtrx,4,1) # i:upadte j:candidate state f:forget o:output
    c=sigmoid(f+f_b_)*c+sigmoid(i)*np.tanh(j)
    h=sigmoid(o)*np.tanh(c)
    print(h)

[[-0.35715187  0.05700316 -0.05667171]]
[[-0.91388124  0.00519003 -0.06978402]]
[[-9.9000210e-01  3.2266005e-04 -3.6948014e-02]]
[[-9.9868983e-01  2.6143445e-05 -1.1944795e-02]]
[[-9.9982423e-01  2.1848582e-06 -3.3091246e-03]]
[[-9.9997628e-01  1.8182082e-07 -8.7816088e-04]]
[[-9.9999690e-01  1.5092073e-08 -2.3409422e-04]]

my imp

[[-0.3571518   0.05700317 -0.05667171]]
[[-0.91388126  0.00519003 -0.06978402]]
[[-9.90001924e-01  3.22659963e-04 -3.69480047e-02]]
[[-9.98689857e-01  2.61434553e-05 -1.19447827e-02]]
[[-9.99824240e-01  2.18485761e-06 -3.30912757e-03]]
[[-9.99976270e-01  1.81820672e-07 -8.78160882e-04]]
[[-9.99996791e-01  1.50920903e-08 -2.34094276e-04]]


### 2 What should we accomplish when reproduce lstm cell in person?

In [79]:
tf.reset_default_graph()
num_units = 5 # = hidden state size
num_input=9
lstm = tf.nn.rnn_cell.LSTMCell(num_units = num_units)
X = tf.placeholder("float", [batch, timesteps, num_input])
x = tf.unstack(X,1) # the same as [tf.reshape(X,(-1,num_input))]
outputs, states = tf.contrib.rnn.static_rnn(lstm, x, dtype=tf.float32)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(lstm._kernel) # Wi || Wj || Wf || Wo , their shape are the same: [num_input+num_units,num_units]
print(lstm._bias)

<tf.Variable 'rnn/lstm_cell/kernel:0' shape=(14, 20) dtype=float32_ref>
<tf.Variable 'rnn/lstm_cell/bias:0' shape=(20,) dtype=float32_ref>


The lstm cell is defining instance with needed parameter in LSTM, in which one step state transition will be calculated.

[source code for DCRNN](https://github.com/liyaguang/DCRNN/blob/master/model/dcrnn_model.py)